<a href="https://colab.research.google.com/github/Araf076/pattern-201-G3/blob/master/Google%20Colab/TTS-Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Steps

## a) Mount to Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## b) Cloning Github Repository to Drive

In [0]:
!pwd
%cd /content/drive/My Drive/
!pwd

/content
/content/drive/My Drive
/content/drive/My Drive


In [0]:
!git clone https://github.com/Araf076/pattern-201-G3

Cloning into 'pattern-201-G3'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 66 (delta 22), reused 41 (delta 9), pack-reused 0
Unpacking objects: 100% (66/66), done.


## c) Installing Libraries

In [0]:
pwd

'/content'

In [0]:
%cd /content/drive/My Drive/pattern-201-G3/TTS-Pytorch

/content/drive/My Drive/pattern-201-G3/TTS-Pytorch


In [0]:
!pip install -q -r requirements.txt

!pip install tensorflow==2.0.0-alpha0

from tensorboardX import SummaryWriter

     |████████████████████████████████| 204kB 13.8MB/s 


## d) Checking Versions 
(Enable GPU from Runtime)

In [0]:
!python --version

import torch
torch.cuda.get_device_name(0)

Python 3.6.9


'Tesla P4'

# Training/Synthesizing English Text-To-Speech

## Dataset Download & Preprocessing

### Dataset Download (if not done already)

In [0]:
pwd

'/content/drive/My Drive/pattern-201-G3/TTS-Pytorch'

In [0]:
import os
import sys
import torch

from os.path import exists, join, expanduser
from audio import preprocess
from utils import download_file
from datasets.lj_speech import LJSpeech

dataset_file_name = 'LJSpeech-1.1.tar.bz2'

datasets_path = os.path.join('/content/drive/My Drive/pattern-201-G3/TTS-Pytorch', 'datasets')
dataset_path = os.path.join(datasets_path, 'LJSpeech-1.1')

if os.path.isdir(dataset_path) and False:
  print("LJSpeech dataset folder already exists")
  sys.exit(0)

else:
  dataset_file_path = os.path.join(datasets_path, dataset_file_name)

  if not os.path.isfile(dataset_file_path):
    url = "http://data.keithito.com/data/speech/%s" % dataset_file_name
    download_file(url, dataset_file_path)
  else:
    print("'%s' already exists" % dataset_file_name)

downloading http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2...


2622MB [02:46, 15.75MB/s]                          


In [0]:
%cd /content/drive/My Drive/pattern-201-G3/TTS-Pytorch/datasets/

/content/drive/My Drive/pattern-201-G3/TTS-Pytorch/datasets


### Extract the Dataset (if not done already)

In [0]:
print("extracting '%s'..." % dataset_file_name)
!tar xvjf LJSpeech-1.1.tar.bz2

Streaming output truncated to the last 5000 lines.
LJSpeech-1.1/wavs/LJ030-0192.wav
LJSpeech-1.1/wavs/LJ041-0078.wav
LJSpeech-1.1/wavs/LJ045-0249.wav
LJSpeech-1.1/wavs/LJ034-0035.wav
LJSpeech-1.1/wavs/LJ010-0152.wav
LJSpeech-1.1/wavs/LJ036-0174.wav
LJSpeech-1.1/wavs/LJ035-0076.wav
LJSpeech-1.1/wavs/LJ032-0176.wav
LJSpeech-1.1/wavs/LJ046-0113.wav
LJSpeech-1.1/wavs/LJ017-0096.wav
LJSpeech-1.1/wavs/LJ004-0098.wav
LJSpeech-1.1/wavs/LJ010-0147.wav
LJSpeech-1.1/wavs/LJ042-0230.wav
LJSpeech-1.1/wavs/LJ041-0033.wav
LJSpeech-1.1/wavs/LJ045-0229.wav
LJSpeech-1.1/wavs/LJ014-0199.wav
LJSpeech-1.1/wavs/LJ002-0082.wav
LJSpeech-1.1/wavs/LJ006-0055.wav
LJSpeech-1.1/wavs/LJ045-0120.wav
LJSpeech-1.1/wavs/LJ050-0028.wav
LJSpeech-1.1/wavs/LJ045-0215.wav
LJSpeech-1.1/wavs/LJ013-0121.wav
LJSpeech-1.1/wavs/LJ008-0025.wav
LJSpeech-1.1/wavs/LJ005-0240.wav
LJSpeech-1.1/wavs/LJ044-0026.wav
LJSpeech-1.1/wavs/LJ048-0127.wav
LJSpeech-1.1/wavs/LJ006-0195.wav
LJSpeech-1.1/wavs/LJ030-0151.wav
LJSpeech-1.1/wavs/LJ038-0

### After Extraction preprocess:

In [0]:
!pwd
%cd /content/drive/My Drive/pattern-201-G3/TTS-Pytorch/datasets/

/content/drive/My Drive/pattern-201-G3/TTS-Pytorch/datasets
/content/drive/My Drive/pattern-201-G3/TTS-Pytorch/datasets


In [0]:
print("pre processing...")
lj_speech = LJSpeech([])
print(lj_speech)
preprocess(dataset_path, lj_speech)

pre processing...


  0%|          | 0/13100 [00:00<?, ?it/s]

100%|██████████| 13100/13100 [1:19:11<00:00,  1.54it/s]


## Training

In [0]:
%cd /content/drive/My Drive/pattern-201-G3/TTS-Pytorch/

/content/drive/My Drive/pattern-201-G3/TTS-Pytorch


In [0]:
!pip install -q -r requirements.txt
!pip install tensorflow==2.0.0-alpha0

from tensorboardX import SummaryWriter

     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 79.9MB 60kB/s 
     |████████████████████████████████| 3.0MB 45.9MB/s 
     |████████████████████████████████| 419kB 46.5MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
cat hparams.py

"""Hyper parameters."""
__author__ = 'Erdene-Ochir Tuguldur'


class HParams:
    """Hyper parameters"""

    disable_progress_bar = False  # set True if you don't want the progress bar in the console

    logdir = "logdir"  # log dir where the checkpoints and tensorboard files are saved

    # audio.py options, these values are from https://github.com/Kyubyong/dc_tts/blob/master/hyperparams.py
    reduction_rate = 4  # melspectrogram reduction rate, don't change because SSRN is using this rate
    n_fft = 2048 # fft points (samples)
    n_mels = 80  # Number of Mel banks to generate
    power = 1.5  # Exponent for amplifying the predicted magnitude
    n_iter = 50  # Number of inversion iterations
    preemphasis = .97
    max_db = 100
    ref_db = 20
    sr = 22050  # Sampling rate
    frame_shift = 0.0125  # seconds
    frame_length = 0.05  # seconds
    hop_length = int(sr * frame_shift)  # samples. =276.
    win_length = int(sr * frame_length)  # samples. =1102.
    max_N = 180  #

In [0]:
cat train-text2mel.py

#!/usr/bin/env python
"""Train the Text2Mel network. See: https://arxiv.org/abs/1710.08969"""
__author__ = 'Erdene-Ochir Tuguldur'

import sys
import time
import argparse
from tqdm import *

import numpy as np

import torch
import torch.nn.functional as F

# project imports
from models import Text2Mel
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import Text2MelDataLoader

parser = argparse.ArgumentParser(description=__doc__, formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--dataset", required=True, choices=['ljspeech', 'mbspeech'], help='dataset name')
args = parser.parse_args()

if args.dataset == 'ljspeech':
    from datasets.lj_speech import vocab, LJSpeech as SpeechDataset
else:
    from datasets.mb_speech import vocab, MBSpeech as SpeechDataset

use_gpu = torch.cuda.is_available()
print('use_gpu', use_gpu)
if use_gpu:
    torch.bac

### Train Text2Mel

In [0]:
!python train-text2mel.py --dataset=ljspeech

use_gpu True
epoch   0 with lr=1.25e-06
  0% 0/13056 [00:00<?, ?audios/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100% 13056/13056 [11:10<00:00, 23.05audios/s, l1=0.29436, att=0.00107]
train epoch loss 0.295429, step=204, total time elapsed: 0h 11m 11s 
epoch   0 with lr=2.56e-04
100% 64/64 [00:16<00:00,  3.86audios/s]
valid epoch loss 0.272219
epoch   1 with lr=2.56e-04
100% 13056/13056 [02:02<00:00, 42.58audios/s, l1=0.27264, att=0.00018] 
train epoch loss 0.272815, step=408, total time elapsed: 0h 13m 30s 
epoch   1 with lr=5.11e-04
100% 64/64 [00:00<00:00, 107.00audios/s]
valid epoch loss 0.250166
epoch   2 with lr=5.11e-04
100% 13056/13056 [01:53<00:00, 85.17audios/s, l1=0.24705, att=0.00016] 
train epoch loss 0.247214, step=612, total time elapsed: 0h 15m 25s 
epoch   2 with lr=7.66e-04
100% 64/6

In [0]:
import sys
import time
import argparse
from tqdm import *

import torch
import torch.nn.functional as F

# project imports
from models import SSRN
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import SSRNDataLoader

In [0]:
cat train-ssrn.py

#!/usr/bin/env python
"""Train the Text2Mel network. See: https://arxiv.org/abs/1710.08969"""
__author__ = 'Erdene-Ochir Tuguldur'

import sys
import time
import argparse
from tqdm import *

import torch
import torch.nn.functional as F

# project imports
from models import SSRN
from hparams import HParams as hp
from logger import Logger
from utils import get_last_checkpoint_file_name, load_checkpoint, save_checkpoint
from datasets.data_loader import SSRNDataLoader

parser = argparse.ArgumentParser(description=__doc__, formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--dataset", required=True, choices=['ljspeech', 'mbspeech'], help='dataset name')
args = parser.parse_args()

if args.dataset == 'ljspeech':
    from datasets.lj_speech import LJSpeech as SpeechDataset
else:
    from datasets.mb_speech import MBSpeech as SpeechDataset

use_gpu = torch.cuda.is_available()
print('use_gpu', use_gpu)
if use_gpu:
    torch.backends.cudnn.benchmark = True

train_data_l

### Train SSRN

In [0]:
!python train-ssrn.py --dataset=ljspeech

use_gpu True
epoch   0 with lr=5.00e-07
  0% 0/13080 [00:00<?, ?audios/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100% 13080/13080 [15:34<00:00, 18.36audios/s, l1=0.08754]
train epoch loss 0.087538, step=545, total time elapsed: 0h 15m 35s 
epoch   0 with lr=2.73e-04
100% 24/24 [00:21<00:00,  1.13audios/s]
valid epoch loss 0.054152
epoch   1 with lr=2.73e-04
100% 13080/13080 [13:57<00:00, 17.82audios/s, l1=0.06520]
train epoch loss 0.065204, step=1090, total time elapsed: 0h 29m 54s 
epoch   1 with lr=4.79e-04
100% 24/24 [00:11<00:00,  2.05audios/s]
valid epoch loss 0.043755
epoch   2 with lr=4.79e-04
100% 13080/13080 [14:41<00:00, 11.42audios/s, l1=0.05855]
train epoch loss 0.058547, step=1635, total time elapsed: 0h 44m 47s 
epoch   2 with lr=3.91e-04
100% 24/24 [00:12<00:00,  1.99audios/s]
valid epo

## Synthesize

In [0]:
%cd /content/drive/My Drive/pattern-201-G3/TTS-Pytorch
!pwd

/content/drive/My Drive/pattern-201-G3/TTS-Pytorch
/content/drive/My Drive/pattern-201-G3/TTS-Pytorch


In [0]:
import os
from os.path import exists, join, expanduser
import sys
import argparse

import numpy as np
import torch

from tqdm import *
import IPython
from IPython.display import Audio

from hparams import HParams as hp
from audio import save_to_wav
from utils import get_last_checkpoint_file_name, load_checkpoint, save_to_png
from datasets.lj_speech import vocab, get_test_data

from models import Text2Mel
from models import SSRN

In [0]:
#Allowed characters: abcdefghijklmnopqrstuvwxyz'.?
SENTENCES = [
        "I enjoyed the wedding of my brother",
        "our batch is one five three",
        "I hate procastination",
        "ami vaat khai",
        "i want to go to dhaka",
        "this is pattern recognition class",
        "The birch canoe slid on the smooth planks.",
        "tomar nam ki?",
        "It's easy to tell the depth of a well.",
        "Hello, my name is Araf"
]

In [0]:
#loading the trained models

torch.set_grad_enabled(False)

text2mel = Text2Mel(vocab)
#print(text2mel)
text2mel.load_state_dict(torch.load("/content/drive/My Drive/pattern-201-G3/TTS-Pytorch/logdir/ljspeech-text2mel/step-005K.pth")['state_dict'])
text2mel = text2mel.eval()

ssrn = SSRN()
ssrn.load_state_dict(torch.load("/content/drive/My Drive/pattern-201-G3/TTS-Pytorch/logdir/ljspeech-ssrn/step-005K.pth")['state_dict'])
ssrn = ssrn.eval()

In [0]:
cat synthesize.py

In [0]:
#generates samples in drive folder sample
!python synthesize.py --dataset=ljspeech

In [0]:
# synthetize by one by one because there is a batch processing bug!
for i in range(len(SENTENCES)):
    sentence = SENTENCES[i]
    normalized_sentence = "".join([c if c.lower() in vocab else '' for c in sentence])
    print(normalized_sentence)
    
    sentences = [normalized_sentence]
    max_N = len(normalized_sentence)
    L = torch.from_numpy(get_test_data(sentences, max_N))
    zeros = torch.from_numpy(np.zeros((1, hp.n_mels, 1), np.float32))
    Y = zeros
    A = None

    for t in range(hp.max_T):
      _, Y_t, A = text2mel(L, Y, monotonic_attention=True)
      Y = torch.cat((zeros, Y_t), -1)
      _, attention = torch.max(A[0, :, -1], 0)
      attention = attention.item()
      if L[0, attention] == vocab.index('E'):  # EOS
          break

    _, Z = ssrn(Y)

    Y = Y.cpu().detach().numpy()
    A = A.cpu().detach().numpy()
    Z = Z.cpu().detach().numpy()
    
    save_to_png('samples-colab/%d-att.png' % (i + 1), A[0, :, :])
    save_to_png('samples-colab/%d-mel.png' % (i + 1), Y[0, :, :])
    save_to_png('samples-colab/%d-mag.png' % (i + 1), Z[0, :, :])

    save_to_wav(Z[0, :, :].T, 'samples-colab/%d-wav.wav' % (i + 1))
    IPython.display.display(Audio('samples-colab/%d-wav.wav' % (i + 1), rate=hp.sr))

I enjoyed the wedding of my brother


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


our batch is one five three


I hate procastination


ami vaat khai


...

this is pattern recognition class


The birch canoe slid on the smooth planks.


tomar nam ki?


It's easy to tell the depth of a well.


Hello my name is Araf
